In [1]:
import os
import re 
import numpy as np 
import pandas as pd 
import anndata as ad

In [2]:
#
# Read in TMA cell data.
#
data_dir = './TMA1_004/TMA1_004_h5ad'

# Load the data.
adatas = {}

# iterate through the file names in the input folder 
for fh in os.listdir(data_dir):

    # ignore any hidden or other non data files 
    if fh.endswith('h5ad'):

        # isolate the core name from the file name using a series of regular expressions
        # the expressions might have to change depending on which slide you use 
        initial_split = re.split('\.', fh)[0]
        core_ID = re.split('-', initial_split)[-1]

        # read the file as anndata object, save in dictionary, {core_ID : adata}
        adatas[core_ID] = ad.read_h5ad(os.path.join(data_dir, fh))

/Users/goecksj/opt/miniconda3/envs/tma-ml/lib/python3.9/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/goecksj/opt/miniconda3/envs/tma-ml/lib/python3.9/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/goecksj/opt/miniconda3/envs/tma-ml/lib/python3.9/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/goecksj/opt/miniconda3/envs/tma-ml/lib/python3.9/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/goecksj/opt/miniconda3/envs/tma-ml/lib/python3.9/site-packages/anndat

In [3]:
# 
# Create dataframe of community spots using BallTree. A community spot is a single cell and its neighbors within a given radius.
# Create spots by averaging all cells within a given radius and replacing single-cell values with spot averages.
#
import pandas as pd
from sklearn.neighbors import BallTree

for core, adata in adatas.items():
    # adata_copy = adata.copy()
    # Build the BallTree
    cell_locs = adata.obs[['X_centroid', 'Y_centroid']].values
    tree = BallTree(cell_locs)

    # Create community spot for each cell.
    communities = np.array([])
    for index, cell_loc in enumerate(cell_locs):
        # print (f"Processing cell {index} with location {cell_loc}")
        indices = tree.query_radius([cell_loc], r=30)[0]
        # if len(indices) < 5:
        #     continue
        # print(f"Found {len(indices)} cells within distance 30 from cell {index}")
        community = adata.X[indices]
        c_mean = community.mean(axis=0)
        # print(f"Community mean:")
        # print(c_mean)
        adata.X[index] = c_mean
    #     break
    # break

In [4]:
#
# Read in TMA metadata.
#
metadata_df = pd.read_csv('./metadata.tsv', sep='\t')
metadata_df

,Core,Subtype
0,G3,TNBC
1,G7,TNBC
2,F9,TNBC
3,F1,TNBC
4,E6,TNBC
5,A8,TNBC
6,A1,Luminal A
7,G5,Luminal A
8,E4,Luminal A
9,H4,Luminal B Her2Pos


In [5]:
#
# Create single data frame with cell features and metadata.
#

# Select cores that are in the metadata.
selected_adatas = {key: value for key, value in adatas.items() if key in metadata_df['Core'].values}
selected_adatas.keys()

# Combine the selected cores into one anndata object.
combined = ad.concat(selected_adatas, label = 'Core')

/Users/goecksj/opt/miniconda3/envs/tma-ml/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [6]:
# Create final dataframe.
final_df = combined.to_df()

# Filter out Control/DNA/AF columns.
marker_cols = final_df.filter(regex="^(?!(Control|DNA|AF))").columns
final_df = final_df[marker_cols]

# Add Core column.
final_df['Core'] = combined.obs['Core']

# Join dataframe with metadata.
final_df = final_df.merge(metadata_df, left_on='Core', right_on='Core')
final_df

,CD3,pERK,Rad51,CCND1,Vimentin,aSMA,Ecad,ER,PR,EGFR,...,CK19,CK17,LaminABC,AR,H2Ax,PCNA,PanCK,CD31,Core,Subtype
0,1.108049,2.367280,1.205983,0.809805,0.643100,0.756149,0.904068,1.873106,0.628908,0.905082,...,1.842349,0.957701,0.734693,1.192826,1.014207,1.136202,1.540347,1.503884,G5,Luminal A
1,1.307919,2.673018,2.009125,0.797774,0.647592,0.979634,1.032403,2.154104,0.505861,0.700817,...,1.749563,1.031223,0.736318,0.864424,0.806115,1.092113,1.198154,1.225833,G5,Luminal A
2,1.054623,2.795245,2.421316,1.015185,0.931346,1.031857,1.408200,3.027466,1.044138,0.778243,...,4.079001,0.987241,1.609144,1.177446,0.864178,1.413665,4.223844,1.306917,G5,Luminal A
3,1.152770,2.482181,2.171659,0.934906,0.853905,0.925194,1.392816,2.705026,1.053693,0.807115,...,3.877788,1.148226,1.742102,1.169645,0.947063,1.488745,3.907415,1.244936,G5,Luminal A
4,1.150260,2.599461,2.265162,0.941080,0.820489,0.918176,1.474642,2.695740,0.983477,0.805180,...,3.973924,1.087175,1.697020,1.180142,0.915597,1.508878,4.092171,1.195510,G5,Luminal A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250675,1.031468,2.284209,2.773331,0.715975,1.057219,1.417538,0.649887,1.515296,0.497423,0.609564,...,1.480717,1.021959,1.099406,0.752269,1.084960,0.629628,0.696489,1.331164,E4,Luminal A
250676,0.950172,2.478513,2.767480,0.671320,0.579839,0.454219,0.651589,1.348625,0.316369,0.656191,...,1.915687,0.676263,0.817493,0.705543,0.761638,0.726469,1.102980,0.922133,E4,Luminal A
250677,1.298196,2.273686,2.790401,1.161770,1.329896,2.153974,0.913626,2.115512,0.501477,0.744121,...,2.825273,0.843333,1.650505,0.991032,0.969978,1.130313,2.273240,1.397758,E4,Luminal A
250678,1.325418,2.613816,2.829215,1.128946,1.215942,1.586671,0.893610,2.039082,0.460092,0.754395,...,2.957357,0.838415,1.302335,0.997982,0.952299,1.056946,2.371508,1.282933,E4,Luminal A


In [7]:
# Write dataframe to file.
final_df.to_csv('./tma_communities.tsv', index=False, sep='\t')